# Gradient Checking

## 1 梯度检查怎么工作的？

检查 $$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$


## 2 一维梯度检查

   ### x ——>  计算函数J ——> J(θ) ——> 计算J的梯度 ——> $\frac{\partial J}{\partial \theta}$

In [1]:
def forward_propagation(x, theta):
    J = theta * x
    return J

In [2]:
def backward_propagation(x, theta):
    dtheta = x
    return dtheta

展示 backward_propagation() 函数正确的在计算梯度$\frac{\partial J}{\partial \theta}$。

- 首先计算“梯度估计” 用个很小的值$\varepsilon$.
    1. $\theta^{+} = \theta + \varepsilon$
    2. $\theta^{-} = \theta - \varepsilon$
    3. $J^{+} = J(\theta^{+})$
    4. $J^{-} = J(\theta^{-})$
    5. $gradapprox = \frac{J^{+} - J^{-}}{2  \varepsilon}$
- 使用backward propagation计算梯度, 并把结果储存到变量 "grad"里面。
- 最后, 用下面公式计算"gradapprox" 和 "grad"的相对不同之处:
$$ difference = \frac {\mid\mid grad - gradapprox \mid\mid_2}{\mid\mid grad \mid\mid_2 + \mid\mid gradapprox \mid\mid_2} \tag{2}$$
计算此公式你需要三步:
   - 1'. 用 np.linalg.norm(...) 计算分子
   - 2'. 计算分母. 你需要两次使用np.linalg.norm(...)。
   - 3'. 除它们.
- If this difference is small (say less than $10^{-7}$), you can be quite confident that you have computed your gradient correctly. Otherwise, there may be a mistake in the gradient computation. 

In [3]:
def gradient_check(x, theta, epsilon = 1e-7):    

    thetaplus = theta+epsilon                               # Step 1
    thetaminus = theta-epsilon                              # Step 2
    J_plus = forward_propagation(x,thetaplus)               # Step 3
    J_minus = forward_propagation(x, thetaminus)            # Step 4
    gradapprox = (J_plus - J_minus)/(2*epsilon)             # Step 5
    
    # 计算grad
    grad = backward_propagation(x, theta)
    
    numerator = np.linalg.norm(grad - gradapprox)                               # Step 1'
    denominator = np.linalg.norm(grad)+np.linalg.norm(gradapprox)               # Step 2'
    difference = numerator/denominator                                          # Step 3'
    
    if difference < 1e-7:
        print ("The gradient is correct!")
    else:
        print ("The gradient is wrong!")
    
    return difference

## 3 N维神经网络的梯度检查

### 首先你正常执行了前向后向传播算法

In [4]:
def forward_propagation_n(X, Y, parameters):
    m = X.shape[1]
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]

    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)

    # Cost
    logprobs = np.multiply(-np.log(A3),Y) + np.multiply(-np.log(1 - A3), 1 - Y)
    cost = 1./m * np.sum(logprobs)
    
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return cost, cache

In [5]:
def backward_propagation_n(X, Y, cache):
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T)
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims = True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T) * 2
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 4./m * np.sum(dZ1, axis=1, keepdims = True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

### 在获得了加过之后你并不清楚模型是否准确现在要进行梯度检验

把W1,b1,W2,b2,...,WL,bL都变成θ大向量。
因为Wl,bl是矩阵，先要把它展平。

J(W1,b1,W2,b2,...,WL,bL) = J(θ)

dW1,db1,dW2,db2,...,dWL,dbL => dθ

For each i in num_parameters:
- To compute `J_plus[i]`:
    1. Set $\theta^{+}$ to `np.copy(parameters_values)`
    2. Set $\theta^{+}_i$ to $\theta^{+}_i + \varepsilon$
    3. Calculate $J^{+}_i$ using to `forward_propagation_n(x, y, vector_to_dictionary(`$\theta^{+}$ `))`.     
- To compute `J_minus[i]`: do the same thing with $\theta^{-}$
- Compute $gradapprox[i] = \frac{J^{+}_i - J^{-}_i}{2 \varepsilon}$

得到个向量 gradapprox, gradapprox[i] 是一个对应 `parameter_values[i]`的梯度的近似值. 你现在能够对比gradapprox向量与后向传播的梯度向量了。 compute: 
$$ difference = \frac {\| grad - gradapprox \|_2}{\| grad \|_2 + \| gradapprox \|_2 } \tag{3}$$

In [6]:
def gradient_check_n(parameters, gradients, X, Y, epsilon = 1e-7):
    parameters_values, _ = dictionary_to_vector(parameters)  # 把参数转换为向量形式
    grad = gradients_to_vector(gradients)                    # 把梯度转换为向量形式
    num_parameters = parameters_values.shape[0]              # 参数向量的维度
    J_plus = np.zeros((num_parameters, 1))                   # J+初始化
    J_minus = np.zeros((num_parameters, 1))                  # J-初始化
    gradapprox = np.zeros((num_parameters, 1))               # 近似梯度初始化
    
    # Compute gradapprox
    for i in range(num_parameters):
        
        # Compute J_plus[i]. Inputs: "parameters_values, epsilon". Output = "J_plus[i]".
        # "_" is used because the function you have to outputs two parameters but we only care about the first one
        ### START CODE HERE ### (approx. 3 lines)
        thetaplus = np.copy(parameters_values)                                      # Step 1
        thetaplus[i][0] = thetaplus[i][0]+epsilon                                   # Step 2
        J_plus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaplus)) # 用比theta大一点点的参数求出J+                                  # Step 3
        ### END CODE HERE ###
        
        # Compute J_minus[i]. Inputs: "parameters_values, epsilon". Output = "J_minus[i]".
        ### START CODE HERE ### (approx. 3 lines)
        thetaminus = np.copy(parameters_values)                                     # Step 1
        thetaminus[i][0] = thetaminus[i][0]-epsilon                               # Step 2        
        J_minus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaminus))   # 用比theta小一点点的参数求出J-                               # Step 3
        
        gradapprox[i] = (J_plus[i]-J_minus[i])/(2*epsilon)
        
    # Compare gradapprox to backward propagation gradients by computing difference.
    numerator = np.linalg.norm(grad - gradapprox)                                           # Step 1'
    denominator = np.linalg.norm(grad)+np.linalg.norm(gradapprox)                           # Step 2'
    difference = numerator/denominator                                                      # Step 3'
  
    if difference > 2e-7:
        print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
    else:
        print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")
    
    return difference